<a href="https://colab.research.google.com/github/Danielon05423/Intro-To-Optimazation/blob/main/Q1_Intro_To_Opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install PyMuPDF
!pip install numpy

In [46]:
import fitz
import re
import numpy as np
from sympy import symbols, Eq, solve
import pandas as pd
from itertools import combinations

In [47]:
#@title main code
def parse_matrix(text, label = "MATRIX"):
    match = re.search(rf"{label}:\s*([0-9,\-;\s]+)", text)
    raw = match.group(1)

    rows = raw.strip().split(';')
    matrix = []
    for row in rows:
            row = row.strip()
            if not row:
                continue
            values = [int(x.strip()) for x in row.split(',') if x.strip()]
            matrix.append(values)

    m = np.array(matrix)
    return m

def parse_vector(text, label = "FUNCTION"):
    match = re.search(rf"{label}:\s*([0-9,\-]+)", text)
    raw = match.group(1)

    vector = [int(x.strip()) for x in raw.strip().split(',') if x.strip()]
    v = np.array(vector)
    return v

def add_artificial_variables(matrix):
    """
    Add artificial variables to the constraint matrix.
    For a 5x8 matrix, this will create a 5x13 matrix where:
    - Original 8 columns remain
    - 5 new columns are added (one for each constraint/row)
    - Each new column has exactly one '1' in its corresponding row

    Args:
        matrix: Original constraint matrix (5x9 including RHS)

    Returns:
        Extended matrix with artificial variables (5x14 including RHS)
    """
    # Separate the constraint coefficients from RHS
    A = matrix[:, :-1]  # 5x8 coefficient matrix
    b = matrix[:, -1:]  # 5x1 RHS vector

    # Create identity matrix for artificial variables (5x5)
    I = np.eye(len(matrix))

    # Combine: [A | I | b] to get 5x14 matrix
    extended_matrix = np.hstack([A, I, b])

    return extended_matrix

def simplex_method(tableau, is_maximization=True):
    """
    Solves linear programming problems using the Simplex Method.

    Args:
        tableau: 2D list representing the initial simplex tableau
        is_maximization: True for maximization, False for minimization

    Returns:
        Final tableau after optimization
    """
    tableau = [row[:] for row in tableau]  # Create a copy
    step = 0

    # Initialize basis - artificial variables s1, s2, s3, s4, s5 are initially basic
    basis = ['x9', 'x10', 'x11', 'x12', 'x13']  # Initial basic variables
    variable_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13']

    print("=== SIMPLEX METHOD SOLVER ===")
    print(f"Problem type: {'Maximization' if is_maximization else 'Minimization'}")
    print("=" * 50)

    while True:
        print(f"\n--- Step {step} ---")
        print_tableau(tableau, basis)

        # Find pivot column
        last_row = tableau[-1][:-1]  # Exclude RHS column

        if is_maximization:
            # For maximization: stop when all coefficients in objective row are non-negative
            if all(val >= 0 for val in last_row):
                print("✅ OPTIMAL SOLUTION REACHED!")
                break
            pivot_col = list(last_row).index(min(last_row))
            print(f"🎯 Pivot column: {pivot_col} ({variable_names[pivot_col]}) (most negative value = {min(last_row):.3f})")
        else:
            # For minimization: stop when all coefficients in objective row are non-positive
            if all(val <= 0 for val in last_row):
                print("✅ OPTIMAL SOLUTION REACHED!")
                break
            pivot_col = list(last_row).index(max(last_row))
            print(f"🎯 Pivot column: {pivot_col} ({variable_names[pivot_col]}) (most positive value = {max(last_row):.3f})")

        # Find pivot row using minimum ratio test
        ratios = []
        for i in range(len(tableau) - 1):  # Exclude objective row
            element = tableau[i][pivot_col]
            if element > 0:
                ratio = tableau[i][-1] / element  # RHS / pivot column element
                ratios.append(ratio)
            else:
                ratios.append(float('inf'))

        if all(r == float('inf') for r in ratios):
            print("❌ UNBOUNDED SOLUTION!")
            return None

        pivot_row = ratios.index(min(ratios))
        print(f"🎯 Pivot row: {pivot_row} ({basis[pivot_row]}) (minimum ratio = {ratios[pivot_row]:.3f})")

        pivot_element = tableau[pivot_row][pivot_col]
        print(f"🎯 Pivot element: {pivot_element:.3f}")

        # Update basis: entering variable replaces leaving variable
        entering_var = variable_names[pivot_col]
        leaving_var = basis[pivot_row]
        basis[pivot_row] = entering_var
        print(f"🔄 Basis change: {leaving_var} leaves, {entering_var} enters")
        print(f"📋 Current basis: {basis}")

        # Perform pivot operation
        print(f"\n⚙️  Performing pivot operation...")

        # Step 1: Normalize pivot row
        tableau[pivot_row] = [x / pivot_element for x in tableau[pivot_row]]

        # Step 2: Eliminate other rows
        for i in range(len(tableau)):
            if i != pivot_row:
                row_factor = tableau[i][pivot_col]
                tableau[i] = [a - row_factor * b for a, b in zip(tableau[i], tableau[pivot_row])]

        step += 1

        # Safety check to prevent infinite loops
        if step > 50:
            print("⚠️  Maximum iterations reached!")
            break

    print(f"\n🏁 FINAL TABLEAU:")
    print_tableau(tableau, basis)
    extract_solution(tableau, is_maximization, basis)

    return tableau


def print_tableau(tableau, basis=None):
    """Print the tableau in a formatted way with current basis"""
    if basis is None:
        basis = ['x9', 'x10', 'x11', 'x12', 'x13']  # Default basis

    print("\nTableau:")
    print("Basic Var |", end="")
    variable_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13']
    for name in variable_names:
        print(f" {name:>8}", end="")
    print(" | RHS")
    print("-" * (len(variable_names) * 9 + 25))

    for i, row in enumerate(tableau):
        if i == len(tableau) - 1:
            print("    z     |", end="")
        else:
            print(f"{basis[i]:>9} |", end="")

        for j, val in enumerate(row):
            if j == len(row) - 1:
                print(f" | {val:8.3f}")
            else:
                print(f" {val:8.3f}", end="")
    print("-" * (len(variable_names) * 9 + 25))


def extract_solution(tableau, is_maximization, basis):
    """Extract and display the solution from the final tableau"""
    print("\n📊 SOLUTION ANALYSIS:")
    print("=" * 30)

    # Get number of original variables (columns - 1 for RHS)
    num_vars = len(tableau[0]) - 1
    solution = [0] * num_vars
    variable_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13']

    # Find basic variables from the basis
    print("Basic variables and their values:")
    for i, basic_var in enumerate(basis):
        value = tableau[i][-1]  # RHS value
        print(f"  {basic_var} = {value:.3f}")

        # Update solution array
        if basic_var in variable_names:
            var_index = variable_names.index(basic_var)
            solution[var_index] = value

    print("\nAll variable values:")
    for i, val in enumerate(solution):
        var_name = variable_names[i]
        print(f"  {var_name} = {val:.3f}")

    # Display objective function value
    obj_value = tableau[-1][-1]  # Last element of objective row

    print(f"\nObjective function value:")
    print(f"  z = {obj_value:.3f}")

    # Print coordinate vector for the first 8 variables
    coordinate_vector = [solution[i] for i in range(8)]
    coordinate_str = ", ".join([f"{val:.3f}" for val in coordinate_vector])
    print(f"\nThe final coordinate vector is: ({coordinate_str})")

In [48]:
file_name = "Work_3.pdf"

doc = fitz.open(file_name)
text = "".join(page.get_text() for page in doc)

m = parse_matrix(text, "MATRIX")
v = parse_vector(text, "FUNCTION")

print('Original Matrix A = \n', m[:,:-1])
print('Original Vector b =', m[:,-1])
print('Function f =', v)

# Add artificial variables to create 5x13 matrix (5x14 with RHS)
m_extended = add_artificial_variables(m)

print('\nExtended Matrix with Artificial Variables:')
print('Shape:', m_extended.shape)
print('Matrix A_extended = \n', m_extended[:,:-1])  # Show coefficients
print('Vector b =', m_extended[:,-1])  # Show RHS

Original Matrix A = 
 [[ 2  8  7  3  4  6  3  1]
 [ 1  8 11  2  9  1  2  7]
 [ 4 12 15  8 15  3  8 14]
 [ 3  4  4  6  6  2  6  7]
 [ 3  4  0  7  1  5  7  1]]
Original Vector b = [5 6 9 3 4]
Function f = [ 2 -6 -2  4  8 -3  6 -3]

Extended Matrix with Artificial Variables:
Shape: (5, 14)
Matrix A_extended = 
 [[ 2.  8.  7.  3.  4.  6.  3.  1.  1.  0.  0.  0.  0.]
 [ 1.  8. 11.  2.  9.  1.  2.  7.  0.  1.  0.  0.  0.]
 [ 4. 12. 15.  8. 15.  3.  8. 14.  0.  0.  1.  0.  0.]
 [ 3.  4.  4.  6.  6.  2.  6.  7.  0.  0.  0.  1.  0.]
 [ 3.  4.  0.  7.  1.  5.  7.  1.  0.  0.  0.  0.  1.]]
Vector b = [5. 6. 9. 3. 4.]


In [49]:
# ===================================================
# 🔼 MAXIMIZATION PHASE
# ===================================================

# Objective function: Maximize f = 2x1 - 6x2 - 2x3 + 4x4 + 8x5 - 3x6 + 6x7 - 3x8
# Extended objective function includes coefficients for artificial variables (typically 0)
z_row_max = [-2, 6, 2, -4, -8, 3, -6, 3, 0, 0, 0, 0, 0, 0]  # 8 original + 5 artificial + RHS

# Create the complete tableau
tableau_max = np.vstack([m_extended, z_row_max])

# Updated column names for the extended tableau
columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'Sol']
index = ['C1', 'C2', 'C3', 'C4', 'C5', 'Z']

print("\n🔼 Initial Extended Tableau for Maximization:")
df_max_init = pd.DataFrame(tableau_max, columns=columns, index=index)
print(df_max_init)

final_max = simplex_method(tableau_max, True)


🔼 Initial Extended Tableau for Maximization:
     x1    x2    x3   x4    x5   x6   x7    x8   x9  x10  x11  x12  x13  Sol
C1  2.0   8.0   7.0  3.0   4.0  6.0  3.0   1.0  1.0  0.0  0.0  0.0  0.0  5.0
C2  1.0   8.0  11.0  2.0   9.0  1.0  2.0   7.0  0.0  1.0  0.0  0.0  0.0  6.0
C3  4.0  12.0  15.0  8.0  15.0  3.0  8.0  14.0  0.0  0.0  1.0  0.0  0.0  9.0
C4  3.0   4.0   4.0  6.0   6.0  2.0  6.0   7.0  0.0  0.0  0.0  1.0  0.0  3.0
C5  3.0   4.0   0.0  7.0   1.0  5.0  7.0   1.0  0.0  0.0  0.0  0.0  1.0  4.0
Z  -2.0   6.0   2.0 -4.0  -8.0  3.0 -6.0   3.0  0.0  0.0  0.0  0.0  0.0  0.0
=== SIMPLEX METHOD SOLVER ===
Problem type: Maximization

--- Step 0 ---

Tableau:
Basic Var |       x1       x2       x3       x4       x5       x6       x7       x8       x9      x10      x11      x12      x13 | RHS
----------------------------------------------------------------------------------------------------------------------------------------------
       x9 |    2.000    8.000    7.000    3.000    4.0

In [50]:
# ===================================================
# 🔽 MINIMIZATION PHASE
# ===================================================

# For minimization, we use the non-negated objective function
# Extended objective function includes coefficients for artificial variables (typically 0)
z_row_min = [-2, 6, 2, -4, -8, 3, -6, 3, 0, 0, 0, 0, 0, 0]  # 8 original + 5 artificial + RHS

# Create the complete tableau using the extended matrix
tableau_min = np.vstack([m_extended, z_row_min])

print("\n🔽 Initial Extended Tableau for Minimization:")
df_min_init = pd.DataFrame(tableau_min, columns=columns, index=index)
print(df_min_init)

final_min = simplex_method(tableau_min, False)


🔽 Initial Extended Tableau for Minimization:
     x1    x2    x3   x4    x5   x6   x7    x8   x9  x10  x11  x12  x13  Sol
C1  2.0   8.0   7.0  3.0   4.0  6.0  3.0   1.0  1.0  0.0  0.0  0.0  0.0  5.0
C2  1.0   8.0  11.0  2.0   9.0  1.0  2.0   7.0  0.0  1.0  0.0  0.0  0.0  6.0
C3  4.0  12.0  15.0  8.0  15.0  3.0  8.0  14.0  0.0  0.0  1.0  0.0  0.0  9.0
C4  3.0   4.0   4.0  6.0   6.0  2.0  6.0   7.0  0.0  0.0  0.0  1.0  0.0  3.0
C5  3.0   4.0   0.0  7.0   1.0  5.0  7.0   1.0  0.0  0.0  0.0  0.0  1.0  4.0
Z  -2.0   6.0   2.0 -4.0  -8.0  3.0 -6.0   3.0  0.0  0.0  0.0  0.0  0.0  0.0
=== SIMPLEX METHOD SOLVER ===
Problem type: Minimization

--- Step 0 ---

Tableau:
Basic Var |       x1       x2       x3       x4       x5       x6       x7       x8       x9      x10      x11      x12      x13 | RHS
----------------------------------------------------------------------------------------------------------------------------------------------
       x9 |    2.000    8.000    7.000    3.000    4.0

In [51]:
import numpy as np
from itertools import combinations

def get_independent_rows(matrix):
    """
    Automatically detects rank and returns a matrix containing only linearly independent rows.
    """
    A = matrix[:, :-1]
    full_rank = np.linalg.matrix_rank(A)
    selected_rows = []

    for i in range(A.shape[0]):
        test_rows = selected_rows + [i]
        A_sub = A[test_rows, :]
        if np.linalg.matrix_rank(A_sub) > len(selected_rows):
            selected_rows.append(i)
        if len(selected_rows) == full_rank:
            break

    all_rows = set(range(matrix.shape[0]))
    removed_rows = sorted(all_rows - set(selected_rows))

    print(f" Detected rank: {full_rank}")
    print(f" Keeping independent rows: {selected_rows}")
    print(f" Removing dependent rows: {removed_rows}")
    return matrix[selected_rows, :], full_rank

def find_extremum_points(matrix, rank):
    """
    Finds all basic feasible solutions (extremum points) using 'rank' variables and equations.
    """
    A = matrix[:, :-1]
    b = matrix[:, -1]
    num_eqs, num_vars = A.shape

    print(f" Reduced matrix dimensions: {num_eqs} equations × {num_vars} variables")
    print(f" Using combinations of {rank} variables")

    extremum_points = []

    for basis in combinations(range(num_vars), rank):
        try:
            A_sub = A[:, basis]
            x_sub = np.linalg.solve(A_sub, b)
            x = np.zeros(num_vars)
            for i, var_index in enumerate(basis):
                x[var_index] = x_sub[i]
            if np.all(x >= 0):
                extremum_points.append(x)
        except np.linalg.LinAlgError:
            continue

    extremum_points.sort(key=lambda v: tuple(v))
    return extremum_points


# === MAIN EXECUTION ===
reduced_m, rank = get_independent_rows(m)
extremum_points = find_extremum_points(reduced_m, rank)

if extremum_points:
    print("\n✅ Extremum Points Found:")
    for i, point in enumerate(extremum_points, 1):
        formatted = ", ".join([f"{x:.3f}" for x in point])
        print(f"Point {i}: ({formatted})")
else:
    print("\n❌ No feasible extremum points found.")


🔍 Detected rank: 4
✅ Keeping independent rows: [0, 1, 2, 4]
❌ Removing dependent rows: [3]
📏 Reduced matrix dimensions: 4 equations × 8 variables
🔢 Using combinations of 4 variables

❌ No feasible extremum points found.
